In [ ]:
#From https://www.tensorflow.org/tutorials/keras/classification

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras

# 헬퍼(helper) 라이브러리를 임포트합니다
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(786, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy', # Sparse category
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, epochs=50, validation_split=0.2)

In [ ]:
model.get_

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
import datetime

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

callbacks = [
  # `val_loss`가 2번의 에포크에 걸쳐 향상되지 않으면 훈련을 멈춥니다.
  tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
  # `./logs` 디렉토리에 텐서보드 로그를 기록니다.
  tf.keras.callbacks.TensorBoard(log_dir=log_dir)
]
history = model.fit(train_images, train_labels,  batch_size=32, epochs=50, callbacks=callbacks, 
                    validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs/fit --port 8089

In [ ]:
import os
os.mkdir("model")

In [ ]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy', # Sparse category
              metrics=['accuracy'])

In [ ]:
filepath = os.path.join("model", "saved-model-{epoch:02d}-{val_loss:.2f}.hdf5")
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath, monitor="val_loss", verbose=0, 
    save_best_only=True, 
    save_weights_only=False, mode='auto', period=1)
csv_logger = keras.callbacks.CSVLogger('training.log')

callbacks = [
  # `val_loss`가 2번의 에포크에 걸쳐 향상되지 않으면 훈련을 멈춥니다.
  tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
  checkpoint, csv_logger
]
history = model.fit(train_images, train_labels, 
                    batch_size=32, epochs=50, callbacks=callbacks,
                    validation_split=0.2)

In [ ]:
model_path = os.path.join("model", "saved-model-01-0.36.hdf5")
new_model = tf.keras.models.load_model(model_path)

In [ ]:
predictions = new_model.predict(test_images)
np.argmax(predictions[0])

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions,  test_labels)
plt.show()

In [ ]:
json_string = new_model.to_json()
json_string

In [ ]:
import json
import pprint
pprint.pprint(json.loads(json_string))